In [1]:
!nvidia-smi 

Wed Dec  4 22:59:54 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off | 00000000:01:00.0 Off |                    0 |
| N/A   35C    P0              71W / 500W |      4MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [3]:
import torch
import numpy as np
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
)
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model,TaskType
from sklearn.metrics import accuracy_score, f1_score
import wandb
import json
from tqdm import tqdm
from typing import Dict

# # Set device and random seeds for reproducibility
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)
np.random.seed(42)

In [5]:
torch.cuda.empty_cache()
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled = True

In [ ]:
class MCDropoutMistral(AutoModelForCausalLM):
    def __init__(self, config):
        super().__init__(config)
        self.dropout_rate = 0.1
        self.to(device)
        
        # Enable dropout by default for all dropout layers
        for module in self.modules():
            if isinstance(module, torch.nn.Dropout):
                module.p = self.dropout_rate
                module.train()
    
    def forward(self, *args, **kwargs):
        try:
            args = tuple(arg.to(device) if torch.is_tensor(arg) else arg for arg in args)
            kwargs = {k: v.to(device) if torch.is_tensor(v) else v for k, v in kwargs.items()}

            # Keep dropout layers in training mode
            for module in self.modules():
                if isinstance(module, torch.nn.Dropout):
                    module.train()

            outputs = super().forward(*args, **kwargs)
            torch.cuda.empty_cache()  # Clear cache after forward pass
            return outputs
        except Exception as e:
            print(f"Forward pass error: {str(e)}")
            raise

In [ ]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        try:
            inputs = {k: v.to(model.device) if hasattr(v, 'to') else v 
             for k, v in inputs.items()}
            labels = inputs.pop("labels").to(model.device)
            outputs = model(**inputs)
            logits = outputs.logits[:, -1, :]  # Get the last token's logits

            # Create one-hot encoded labels with proper shape
            num_classes = logits.size(-1)
            one_hot_labels = torch.zeros(
                labels.size(0), 
                num_classes, 
                device=labels.device, 
                dtype=torch.float32
            )
            one_hot_labels.scatter_(1, labels.unsqueeze(1), 1)

            # Compute cross entropy loss
            loss = torch.nn.functional.cross_entropy(
                logits,
                labels,
                ignore_index=-100
            )

            with torch.no_grad():
                predictions = torch.argmax(logits, dim=-1)
                valid_mask = labels != -100
                accuracy = (predictions[valid_mask] == labels[valid_mask]).float().mean()

                if self.args.logging_steps > 0 and self.state.global_step % self.args.logging_steps == 0:
                    wandb.log({
                        "train_loss": loss.item(),
                        "train_accuracy": accuracy.item(),
                        "train_step": self.state.global_step,
                        "train_epoch": self.state.epoch,
                    })

            # Clear unnecessary tensors
            del predictions, valid_mask
            torch.cuda.empty_cache()

            return (loss, outputs) if return_outputs else loss

        except Exception as e:
            print(f"Loss computation error: {str(e)}")
            raise

In [8]:
def prepare_and_tokenize_dataset(file_path, tokenizer):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = [json.loads(line) for line in f]

        formatted_data = []
        for item in tqdm(data, desc="Processing dataset"):
            question = f"""Question: {item['question']}
Available options:
A: {item['options']['A']}
B: {item['options']['B']}
C: {item['options']['C']}
D: {item['options']['D']}
E: {item['options']['E']}

Please select the correct answer."""
            
            tokenized = tokenizer(
                question,
                padding="max_length",
                truncation=True,
                max_length=512,
                return_tensors="pt"
            )
            
            if item['answer_idx'] in ['A', 'B', 'C', 'D', 'E']:
                label = ord(item['answer_idx']) - ord('A')
                tokenized["labels"] = torch.tensor(label, dtype=torch.long)
                formatted_data.append({k: v.squeeze() for k, v in tokenized.items()})
            
        return Dataset.from_list(formatted_data)
    except Exception as e:
        print(f"Dataset preparation error: {str(e)}")
        raise

def compute_metrics(eval_preds):
    try:
        logits, labels = eval_preds
        predictions = np.argmax(logits, axis=-1)
        
        mask = labels != -100
        filtered_predictions = predictions[mask]
        filtered_labels = labels[mask]
        
        metrics = {
            "accuracy": float(accuracy_score(filtered_labels, filtered_predictions)),
            "f1": float(f1_score(filtered_labels, filtered_predictions, average="weighted"))
        }
        return metrics
    except Exception as e:
        print(f"Metrics computation error: {str(e)}")
        raise

In [17]:
class CustomDataCollator:
    def __init__(self, tokenizer, max_length=512):
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __call__(self, examples):
        batch = {}
        
        # Process input_ids and attention_mask
        batch_encoding = self.tokenizer.pad(
            [{"input_ids": ex["input_ids"], "attention_mask": ex["attention_mask"]} for ex in examples],
            padding=True,
            return_tensors="pt"
        )
        
        batch["input_ids"] = batch_encoding["input_ids"]
        batch["attention_mask"] = batch_encoding["attention_mask"]
        
        # Process labels
        if "labels" in examples[0]:
            batch["labels"] = torch.tensor([ex["labels"] for ex in examples], dtype=torch.long)
            
        return batch

In [18]:
def tokenize_function(examples: Dict, tokenizer, max_length: int = 512):
    """Tokenize the input data."""
    prompts = [f"{instruction}\n{response}" 
              for instruction, response in zip(examples["instruction"], examples["response"])]
    
    # Tokenize inputs
    tokenized = tokenizer(
        prompts,
        truncation=True,
        max_length=max_length,
        padding="max_length",
    )
    
    # Create labels (same as input_ids for causal language modeling)
    labels = tokenized["input_ids"].copy()
    
    # Convert to PyTorch tensors
    return {
        "input_ids": torch.tensor(tokenized["input_ids"]),
        "attention_mask": torch.tensor(tokenized["attention_mask"]),
        "labels": torch.tensor(labels)
    }

In [ ]:
def prepare_model_and_tokenizer():
    """Initialize the model and tokenizer with LoRA configuration."""
    try:
        tokenizer = AutoTokenizer.from_pretrained(
            "mistralai/Mistral-7B-Instruct-v0.3",
            use_fast=True,
            padding_side="right",
            trust_remote_code=True
        )
        
        model = MCDropoutMistral.from_pretrained(
            "mistralai/Mistral-7B-Instruct-v0.3",
            load_in_8bit=True,
            device_map="auto"
        )
        
        # Prepare model for k-bit training
        model = prepare_model_for_kbit_training(model)
        
        # Define LoRA Config
        lora_config = LoraConfig(
            r=16,  # rank
            lora_alpha=32,
            target_modules=["q_proj", "v_proj"],
            lora_dropout=0.05,
            bias="none",
            task_type=TaskType.CAUSAL_LM
        )
        
        # Get PEFT model
        model = get_peft_model(model, lora_config)
        
        if tokenizer.pad_token is None:
            tokenizer.add_special_tokens({"pad_token": "[PAD]"})
            model.resize_token_embeddings(len(tokenizer))
        
        return model, tokenizer
    
    except Exception as e:
        print(f"Error loading model/tokenizer: {str(e)}")
        raise

In [20]:
# Initialize model and tokenizer
model, tokenizer = prepare_model_and_tokenizer()

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [21]:
# Print trainable parameters info
model.print_trainable_parameters()

trainable params: 6,815,744 || all params: 7,254,847,488 || trainable%: 0.0939


In [22]:
# Prepare datasets
train_dataset = prepare_and_tokenize_dataset("train.jsonl", tokenizer)
val_dataset = prepare_and_tokenize_dataset("dev.jsonl", tokenizer)
print(f"Train samples: {len(train_dataset)}, Validation samples: {len(val_dataset)}")

Processing dataset: 100%|██████████| 1272/1272 [00:00<00:00, 1618.74it/s]


Train samples: 10178, Validation samples: 1272


In [ ]:
training_args = TrainingArguments(
    output_dir="./mistral-args-medical-qa-lora-v5",
    num_train_epochs=2,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    warmup_steps=100,
    weight_decay=0.01,
    logging_steps=10,
    save_strategy="steps",
    save_steps=100,
    learning_rate=2e-4,
    optim="paged_adamw_8bit",
    report_to="wandb",
    remove_unused_columns=False,
    save_total_limit=1,
    metric_for_best_model="accuracy",
    greater_is_better=True
)


In [24]:
# Initialize WandB
wandb.init(
    project="mistral-mc-finetune",
    name="fine-tuning-train-val-only",
    config={
        "model": "Mistral-7B-Instruct-v0.3",
        "epochs": 3,
        "batch_size": 8,
        "learning_rate": 2e-4
    }
)

# Initialize trainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    #eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=CustomDataCollator(tokenizer),
    compute_metrics=compute_metrics
)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [25]:
# Before training starts
import gc
torch.cuda.empty_cache()
gc.collect()

# Train
try:
    trainer.train()
    model.save_pretrained("./llama-medical-qa-lora-v5")
finally:
    torch.cuda.empty_cache()
    gc.collect()
    wandb.finish()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/home/koppisetty.e/.local/lib/python3.8/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  retur

Step,Training Loss
10,16.542000
20,13.106200
30,9.850600
40,6.933000
50,5.605300
60,5.574800
70,5.169300
80,4.851800
90,5.071400
100,4.981800


/home/koppisetty.e/.local/lib/python3.8/site-packages/peft/utils/save_and_load.py:257: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/home/koppisetty.e/.local/lib/python3.8/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/home/koppisetty.e/.local/lib/python3.8/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/ho

train/epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇██████
train/grad_norm,▂▅▄█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/learning_rate,▂▃▄▄▇███▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁
train/loss,█▆▄▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_accuracy,▁▁▁▁▂▁▄▄▂▅▅▄▂█▄▅▅▄▂▇▂▁▅▂▂▇▇▇▅▂▁▁▁▄▇▄▄▅▂▄
train_epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▅▅▆▆▇▇▇▇▇▇███
train_loss,▆▂▂██▆▅▄▆▇█▆▁▅▇▄▆▁▂▂▃█▅▁▁▂▅▆▅█▁█▆▅▆▅▅▇▄▇
train_step,▁▁▁▁▁▂▂▂▂▂▃▃▄▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇████
total_flos,4.450598392619336e+17
train/epoch,1.99843
